In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta
import numpy as np
import calendar
import requests
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### Obtain the Weather Data <br><br>

Source of Data: https://psl.noaa.gov/boulder/data.daily.html <br>
querie code: var i = 1; [].forEach.call(document.querySelectorAll('table'), function(x) { console.log(i++, x); });

In [2]:
df = pd.read_csv('../Dataset/BOULDER_Electric_Vehicle_Charging_Station_Data.csv')

df.Start_Date___Time.min(), df.Start_Date___Time.max()

('1/1/2018 17:49', '9/9/2023 9:19')

In [3]:
# These tables contains the indexes of weather data tables for the period Jan-2018 to Nov-2023
tables_year = {'2018': [num for num in range(195, 218, 2)],
               '2019': [num for num in range(219, 242, 2)],
               '2020': [num for num in range(243, 266, 2)],
               '2021': [num for num in range(267, 290, 2)],
               '2022': [num for num in range(291, 314, 2)],
               '2023': [num for num in range(315, 336, 2)]}

tables_year

{'2018': [195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217],
 '2019': [219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241],
 '2020': [243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265],
 '2021': [267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289],
 '2022': [291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313],
 '2023': [315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335]}

In [4]:
# Source of Implementation: - https://towardsdatascience.com/a-guide-to-scraping-html-tables-with-pandas-and-beautifulsoup-7fc24c331cf7

#                           - https://github.com/SwethaSrikari/Predicting-EV-charging-demand/blob/main/Web_scraping_Colorado_weather.ipynb


# Function to scrape weather data table for a specific year from a given URL
def get_weather_data_table_basedOnYear(url, year):
    # Send a GET request to the specified URL
    page = requests.get(url) 
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(page.text, 'html.parser') 
    
    rows = [] # Initialize an empty list to store rows of data

    # Loop through the tables associated with the specified year
    for table in tables_year[year]: 

        # Loop through the children of the table element
        for i, child in enumerate(soup.find_all('table')[table].children): 
            row = [] # Initialize an empty list to store data for each row

            # Loop through the cells (td elements) in the row
            for td in child: 
                try:
                    row.append(td.text) # Attempt to extract text content from each cell and append to the row list
                except:
                    continue # If an exception occurs (e.g., if the cell is not a td element), continue to the next iteration

            # Check if the row contains any data (i.e., if it's not an empty row)
            if len(row) > 0:
                rows.append(row) # Append the non-empty row to the list of rows

    # Create a DataFrame using the extracted rows, specifying columns and dropping duplicate rows
    df = pd.DataFrame(rows[1:], columns=rows[0]).drop_duplicates(keep=False)

    return df

In [5]:
# Define the url source
url = 'https://psl.noaa.gov/boulder/data.daily.html'

# Take the Weather Data Tables for each year
df_2018 = get_weather_data_table_basedOnYear(url, str(2018))
df_2019 = get_weather_data_table_basedOnYear(url, str(2019))
df_2020 = get_weather_data_table_basedOnYear(url, str(2020))
df_2021 = get_weather_data_table_basedOnYear(url, str(2021))
df_2022 = get_weather_data_table_basedOnYear(url, str(2022))
df_2023 = get_weather_data_table_basedOnYear(url, str(2023))

print('\nShapes of DataFrames:',df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape)


Shapes of DataFrames: (366, 8) (366, 8) (366, 8) (366, 8) (366, 8)


### Prepare the Weather Data <br><br>
Source of the weather data cleaning: https://psl.noaa.gov/boulder/getdata.html

In [6]:
# Counting the number of rows containing the word 'Miss' in multiple DataFrames
for year in range(2018, 2024):
    miss_count = locals()[f'df_{year}'].apply(lambda row: any(row == 'Miss '), axis=1).sum()
    print(f"Number of rows containing 'Miss' in df_{year} are/is: {miss_count}")

Number of rows containing 'Miss' in df_2018 are/is: 1
Number of rows containing 'Miss' in df_2019 are/is: 1
Number of rows containing 'Miss' in df_2020 are/is: 0
Number of rows containing 'Miss' in df_2021 are/is: 1
Number of rows containing 'Miss' in df_2022 are/is: 1
Number of rows containing 'Miss' in df_2023 are/is: 1


After examining data from multiple DataFrames (df_2018 to df_2023), we observed one row containing the value 'Miss' in each DataFrame, outside of df_2020. This occurrence is attributed to leap years, where February 29th is not present, leading to the 'Miss' value in our dataset.

In [7]:
# Function to check for leap years
def is_leap_year(year):
    return calendar.isleap(year)


# Function to remove leading and trailing spaces from a string value
def strip_spaces(value):
    # Check if the value is a string before applying strip()
    return value.strip() if isinstance(value, str) else value



# Function to prepare weather data
def prepare_weather_data(df):
    
    # Clean column names by removing extra spaces
    df.columns = df.columns.str.strip()

    # Remove extra spaces from all rows in the DataFrame
    df = df.applymap(strip_spaces)

    # Handle 'T' as a trace (less than 0.01 inches for precipitation and 0.1 for snow)
    df["Snow"] = df["Snow"].replace('T', 0.099)
    df["Precipitation"] = df["Precipitation"].replace('T', 0.0099)

    # Exclude rows with 'Miss' from the DataFrame
    df = df.loc[~(df == 'Miss').any(axis=1)]

    # Convert year, month, day to numeric
    df[["Year", "Month", "Day"]] = df[["Year", "Month", "Day"]].apply(pd.to_numeric)

    # Handle February based on leap year
    mask = (df["Month"] == 2) & (df["Day"] == 29) & ~df["Year"].apply(is_leap_year)
    # If the mask is true for a row, set the value of the "Day" column to 28
    df.loc[mask, "Day"] = 28

    # Convert specified columns to numeric and remove 'Snow Depth' column
    df = df[["Year", "Month", "Day", "Maximum T", "Minimum T", "Precipitation", "Snow"]].apply(pd.to_numeric)

    # Create a date column
    df["Date"] = pd.to_datetime(df[["Year", "Month", "Day"]]).dt.date.astype("datetime64")

    return df

# Clean the Weather Data Tables for each year
df_2018 = prepare_weather_data(df_2018)
df_2019 = prepare_weather_data(df_2019)
df_2020 = prepare_weather_data(df_2020)
df_2021 = prepare_weather_data(df_2021)
df_2022 = prepare_weather_data(df_2022)
df_2023 = prepare_weather_data(df_2023)

In [8]:
for year in range(2018, 2024):
    miss_count = locals()[f'df_{year}'].apply(lambda row: any(row == 'Miss '), axis=1).sum()
    print(f"Number of rows containing 'Miss' in df_{year} are/is: {miss_count}")

print('\nShapes of DataFrames:',df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape)

Number of rows containing 'Miss' in df_2018 are/is: 0
Number of rows containing 'Miss' in df_2019 are/is: 0
Number of rows containing 'Miss' in df_2020 are/is: 0
Number of rows containing 'Miss' in df_2021 are/is: 0
Number of rows containing 'Miss' in df_2022 are/is: 0
Number of rows containing 'Miss' in df_2023 are/is: 0

Shapes of DataFrames: (365, 8) (365, 8) (366, 8) (365, 8) (365, 8)


In [9]:
# Concat all weather dataframes into one DataFrame
weather_df = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)
weather_df.sort_values('Date', inplace=True)
weather_df['Date'].is_monotonic_increasing

True

In [10]:
len(df_2018) + len(df_2019) + len(df_2020) + len(df_2021) + len(df_2022) + len(df_2023)  == len(weather_df)

True

In [11]:
weather_df

,Year,Month,Day,Maximum T,Minimum T,Precipitation,Snow,Date
0,2018,1,1,30,12,0.0,0.0,2018-01-01
1,2018,1,2,46,12,0.0,0.0,2018-01-02
2,2018,1,3,50,20,0.0,0.0,2018-01-03
3,2018,1,4,52,24,0.0,0.0,2018-01-04
4,2018,1,5,62,25,0.0,0.0,2018-01-05
...,...,...,...,...,...,...,...,...
2155,2023,11,26,38,12,0.0,0.0,2023-11-26
2156,2023,11,27,43,15,0.0,0.0,2023-11-27
2157,2023,11,28,53,24,0.0,0.0,2023-11-28
2158,2023,11,29,52,23,0.0,0.0,2023-11-29


In [12]:
# In https://psl.noaa.gov/boulder/getdata.html tell us Missing values are indicated by -998.0.
# So we count rows where any value in the selected columns is equal to -998
count_missing_values = (weather_df[['Year', 'Month', 'Day', 'Maximum T', 'Minimum T', 'Precipitation', 'Snow']] == -998).any(axis=1).sum()

print(f"Number of rows with value equal to -998.0 are/is: {count_missing_values}")

Number of rows with value equal to -998.0 are/is: 0


In [13]:
weather_df.isna().sum()

Year             0
Month            0
Day              0
Maximum T        0
Minimum T        0
Precipitation    0
Snow             0
Date             0
dtype: int64

In [14]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2160 entries, 0 to 2159
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Year           2160 non-null   int64         
 1   Month          2160 non-null   int64         
 2   Day            2160 non-null   int64         
 3   Maximum T      2160 non-null   int64         
 4   Minimum T      2160 non-null   int64         
 5   Precipitation  2160 non-null   float64       
 6   Snow           2160 non-null   float64       
 7   Date           2160 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 151.9 KB


****

### Merge Weather Data into the BOULDER Electric Vehicle Charging Station DataFrame

In [15]:
df.head()

,ObjectId2,Station_Name,Address,City,State_Province,Zip_Postal_Code,Start_Date___Time,Start_Time_Zone,End_Date___Time,End_Time_Zone,Total_Duration__hh_mm_ss_,Charging_Time__hh_mm_ss_,Energy__kWh_,GHG_Savings__kg_,Gasoline_Savings__gallons_,Port_Type,ObjectID
0,1,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,1/1/2018 17:49,MDT,1/1/2018 19:52,MDT,2:03:02,2:02:44,6.504,2.732,0.816,Level 2,0
1,2,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,1/2/2018 8:52,MDT,1/2/2018 9:16,MDT,0:24:34,0:24:19,2.481,1.042,0.311,Level 2,1
2,3,BOULDER / JUNCTION ST1,2280 Junction Pl,Boulder,Colorado,80301,1/2/2018 21:11,MDT,1/3/2018 6:23,MDT,9:12:21,3:40:52,15.046,6.319,1.888,Level 2,2
3,4,BOULDER / ALPINE ST1,1275 Alpine Ave,Boulder,Colorado,80304,1/3/2018 9:19,MDT,1/3/2018 11:14,MDT,1:54:51,1:54:29,6.947,2.918,0.872,Level 2,3
4,5,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,1/3/2018 14:13,MDT,1/3/2018 14:30,MDT,0:16:58,0:16:44,1.800,0.756,0.226,Level 2,4


In [16]:
df.columns

Index(['ObjectId2', 'Station_Name', 'Address', 'City', 'State_Province',
       'Zip_Postal_Code', 'Start_Date___Time', 'Start_Time_Zone',
       'End_Date___Time', 'End_Time_Zone', 'Total_Duration__hh_mm_ss_',
       'Charging_Time__hh_mm_ss_', 'Energy__kWh_', 'GHG_Savings__kg_',
       'Gasoline_Savings__gallons_', 'Port_Type', 'ObjectID'],
      dtype='object')

In [17]:
df[['Start_Date___Time','End_Date___Time','Total_Duration__hh_mm_ss_','Charging_Time__hh_mm_ss_']].head()

,Start_Date___Time,End_Date___Time,Total_Duration__hh_mm_ss_,Charging_Time__hh_mm_ss_
0,1/1/2018 17:49,1/1/2018 19:52,2:03:02,2:02:44
1,1/2/2018 8:52,1/2/2018 9:16,0:24:34,0:24:19
2,1/2/2018 21:11,1/3/2018 6:23,9:12:21,3:40:52
3,1/3/2018 9:19,1/3/2018 11:14,1:54:51,1:54:29
4,1/3/2018 14:13,1/3/2018 14:30,0:16:58,0:16:44


In [18]:
def date_columns_to_datetime(df, date_columns):
    for column in date_columns:
        df[column] = df[column].apply(lambda x: pd.to_datetime(x, errors='coerce'))

    return df

# Must have as a seperator ':'
def date_columns_to_timedelta(time_str):
    try:
        hours, minutes, seconds = map(int, time_str.split(':'))
        return pd.Timedelta(hours=hours, minutes=minutes, seconds=seconds)
    except ValueError:
        return pd.NaT  # Handle invalid time format  
    
# Make the columns datetimes and timedelta
df1 = df.copy()

date_columns_to_dt= ['Start_Date___Time','End_Date___Time']
df1 = date_columns_to_datetime(df1, date_columns_to_dt)

date_columns_to_td = ['Total_Duration__hh_mm_ss_','Charging_Time__hh_mm_ss_']
for column in date_columns_to_td:
    df1[column] = df1[column].apply(date_columns_to_timedelta)

In [19]:
df1[['Start_Date___Time','End_Date___Time','Total_Duration__hh_mm_ss_','Charging_Time__hh_mm_ss_']].head()

,Start_Date___Time,End_Date___Time,Total_Duration__hh_mm_ss_,Charging_Time__hh_mm_ss_
0,2018-01-01 17:49:00,2018-01-01 19:52:00,0 days 02:03:02,0 days 02:02:44
1,2018-01-02 08:52:00,2018-01-02 09:16:00,0 days 00:24:34,0 days 00:24:19
2,2018-01-02 21:11:00,2018-01-03 06:23:00,0 days 09:12:21,0 days 03:40:52
3,2018-01-03 09:19:00,2018-01-03 11:14:00,0 days 01:54:51,0 days 01:54:29
4,2018-01-03 14:13:00,2018-01-03 14:30:00,0 days 00:16:58,0 days 00:16:44


In [20]:
df1.isna().sum()[df1.isna().sum()>0]

End_Date___Time    4
dtype: int64

In [21]:
# Create a Date column to use it like a foreign key for the weather DataFrames
df1["Date"] = df1['Start_Date___Time'].dt.date.astype("datetime64")

# Sort increasing the dates per stations 
print(df1['Date'].is_monotonic_increasing)
#print(df1.groupby('Station_Name')['Date'].apply(lambda x: x.is_monotonic_increasing).all())

df1.sort_values(by='Date', inplace=True)
print(df1['Date'].is_monotonic_increasing)
#df1.sort_values(by=['Station_Name', 'Date'], inplace=True)
#print(df1.groupby('Station_Name')['Date'].apply(lambda x: x.is_monotonic_increasing).all())

False
True


In [22]:
# Merge the main Dataframe with the Weather Data
print(df1.shape)
df1 = df1.merge(weather_df, on='Date', how='left')
df1.shape

(148136, 18)


(148136, 25)

In [23]:
df1.isna().sum()[df1.isna().sum()>0]

End_Date___Time    4
dtype: int64

****

### Data Cleaning / Preparation Based on paper

- In the Dataset was considered: station ID and location, connection port, start and end times, connection
durations, charging durations, kWh consumed, greenhouse gas reductions and gasoline
savings, and unique driver identification.
- The snow and precipitation was input in millimeters
- Min and Max temperature in Fahrenheit (◦F)
- Weekday is a Categorical variable Mon, Tue, Wed, Thu, Fri, Sat, and Sun
- Month also is a Categorical variable Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, Oct, Nov, and Dec
- All missing and negative data were removed from the dataset

**We are apply this preprocessing rules in entire dataset the paper use 43,659 charging sessions (January 2018 to August 2022)**

The Data Measurements: 
- Temperatures are in degrees Fahrenheit (◦F).
- Precipitation, snowfall and snow depth are in inches
  
**1 inche = 25.4 mm**      |   Source: https://www.unitconverters.net/length/inches-to-mm.htm

In [32]:
def data_transformation(df):

    # Convert inches to millimeters
    df['Snow'] = df['Snow'] * 25.4
    df['Precipitation'] = df['Precipitation'] * 25.4

    # Make the Month and Day to categorical variables 
    df['Weekday'] = df['Date'].dt.day_name()
    df['Month'] = df['Date'].dt.month_name()

    # Remove null (0) values in Engery_kwh_
    df = df[df['Energy__kWh_'] > 0]

    # Change datatype from float to integer
    df[["Year", "Day"]] = df[["Year", "Day"]].astype(np.int64)
    
    return df

df2 = df1.copy()
df2 = data_transformation(df1)
print('We drop the',round((df1.shape[0] - df2.shape[0]) / df1.shape[0],3),'% of the Dataset.')

We drop the 0.109 % of the Dataset.


In [33]:
df2['Energy__kWh_'].value_counts()

0.001     71
0.008     56
0.002     47
0.014     43
0.006     35
          ..
29.271     1
14.800     1
58.241     1
17.243     1
12.436     1
Name: Energy__kWh_, Length: 23040, dtype: int64

In [34]:
df2[['Energy__kWh_','Date','Day','Year','Weekday','Month','Minimum T','Maximum T','Snow','Precipitation']]

,Energy__kWh_,Date,Day,Year,Weekday,Month,Minimum T,Maximum T,Snow,Precipitation
0,6.504,2018-01-01,1,2018,Monday,January,12,30,0.0,0.0
1,6.504,2018-01-01,1,2018,Monday,January,12,30,0.0,0.0
2,15.046,2018-01-02,2,2018,Tuesday,January,12,46,0.0,0.0
3,2.481,2018-01-02,2,2018,Tuesday,January,12,46,0.0,0.0
4,15.046,2018-01-02,2,2018,Tuesday,January,12,46,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
148130,24.374,2023-11-30,30,2023,Thursday,November,22,38,0.0,0.0
148131,8.123,2023-11-30,30,2023,Thursday,November,22,38,0.0,0.0
148132,12.436,2023-11-30,30,2023,Thursday,November,22,38,0.0,0.0
148133,15.692,2023-11-30,30,2023,Thursday,November,22,38,0.0,0.0


### Make the Dataset into Daily, Weekly and Monthly aggregated datasets and keep the features based on paper

In [39]:
# The paper approach duration (January 2018 to August 2022)
print('DataFrame before the filtering:\nStart and End Datetime:',df2.Date.min(), df2.Date.max(),'\nLength:',len(df2),'\n')
df2 = df2[df2['Date'] < '2022-08-01']
df2['Date'] = df2.loc[:, 'Date'].apply(pd.to_datetime)

columns = ['Energy__kWh_','Date','Day','Year','Weekday','Month','Minimum T','Maximum T','Snow','Precipitation']
df2.drop_duplicates(subset=columns, keep='first', inplace=True)
print('DataFrame after the filtering:\nStart and End Datetime:',df2.Date.min(), df2.Date.max(),'\nLength:',len(df2),'\n')

DataFrame before the filtering:
Start and End Datetime: 2018-01-01 00:00:00 2022-08-31 00:00:00 
Length: 41644 

DataFrame after the filtering:
Start and End Datetime: 2018-01-01 00:00:00 2022-07-31 00:00:00 
Length: 39774 



In [40]:
df2[['Energy__kWh_','Date','Day','Year','Weekday','Month','Minimum T','Maximum T','Snow','Precipitation']].head()

,Energy__kWh_,Date,Day,Year,Weekday,Month,Minimum T,Maximum T,Snow,Precipitation
0,6.504,2018-01-01,1,2018,Monday,January,12,30,0.0,0.0
2,15.046,2018-01-02,2,2018,Tuesday,January,12,46,0.0,0.0
3,2.481,2018-01-02,2,2018,Tuesday,January,12,46,0.0,0.0
6,6.947,2018-01-03,3,2018,Wednesday,January,20,50,0.0,0.0
7,1.800,2018-01-03,3,2018,Wednesday,January,20,50,0.0,0.0


In [41]:
df2['Date'].is_monotonic_increasing

True

In [42]:
df2.drop_duplicates(inplace=True)

# Daily dataset - Aggregated by sum
columns_daily = ['Energy__kWh_', 'Weekday', 'Month', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']
daily_df = df2.groupby(['Date', 'Month', 'Weekday'], as_index=False).sum(numeric_only=True)[columns_daily]

# Weekly dataset - Aggregated by sum
columns_weekly = ['Energy__kWh_', 'Day', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']
weekly_df = df2.groupby(pd.Grouper(freq='W', key='Date')).sum(numeric_only=True)[columns_weekly]

# Monthly dataset - Aggregated by sum
columns_monthly = ['Energy__kWh_', 'Month', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']
monthly_df = df2.groupby(['Month', 'Year', ], as_index=False).sum(numeric_only=True)[columns_monthly]


# Evaluate the statistics Based On Paper

# Daily analysis
daily_stats = pd.DataFrame(daily_df['Energy__kWh_'].describe()[['count', 'min', 'max']])
daily_stats.rename(index={'count': 'Number_Of_Records'}, inplace=True)

# Weekly analysis
weekly_stats = pd.DataFrame(weekly_df['Energy__kWh_'].describe()[['count', 'min', 'max']])
weekly_stats.rename(index={'count': 'Number_Of_Records'}, inplace=True)

# Monthly analysis
monthly_stats = pd.DataFrame(monthly_df['Energy__kWh_'].describe()[['count', 'min', 'max']])
monthly_stats.rename(index={'count': 'Number_Of_Records'}, inplace=True)

# Display the results
stats_df = pd.concat([daily_stats, weekly_stats, monthly_stats], axis=1)
stats_df.columns = ['Daily', 'Weekly', 'Monthly']
stats_df.T # Transpose the DataFrame

,Number_Of_Records,min,max
Daily,1668.0,0.749,725.552
Weekly,239.0,144.751,4097.781
Monthly,55.0,1015.725,15489.792


![](../images/Statistical_Summary.jpg) 

In [31]:
# daily_df.to_csv('../Dataset/Boulder_Daily.csv')
# weekly_df.to_csv('../Dataset/Boulder_Weekly.csv')
# monthly_df.to_csv('../Dataset/Boulder_Monthly.csv')